In [108]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
X,labels=read_data('datasets/data sets/MSRC-v1.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()
from models import MSE,MDcR,DSE,MVBG
model_MSE = MSE()
model_MDcR = MDcR()
model_DSE = DSE()
model_MVBG = MVBG(1,1,1)

res_nim = []
for d_ in range(10,12):
    Y = model_MSE.mse(X,gamma = 0.9,d_=d_,t=1e8,epoch=500)
    Y = model_MDcR.mdcr(X,d_=10,t=1e8,lmd=10,epoch=500)
    Y  = model_MVBG.mvbg(X,50,10,500)
    nmi_list = []
    for i in range(20):
        pred = kmeans(Y,7)
        # criterion
        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi = normalized_mutual_info_score(labels,pred)
        nmi_list.append(nmi)
    nmi = np.mean(nmi_list)
    res_nim.append(nmi)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(10,12),res_nim)

In [106]:
#DSE 單獨
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from utils import *
from tqdm import tqdm
X,labels=read_data('datasets/data sets/MSRC-v1.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

from models import DSE
model_DSE = DSE()
nmi_list=[]
for _ in tqdm(range(20)):
    pred = model_DSE.dse(X,7,2,500)
    from sklearn.metrics.cluster import normalized_mutual_info_score
    nmi = normalized_mutual_info_score(labels,pred)
    # print(nmi)
    nmi_list.append(nmi)
print(np.mean(nmi_list))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 20/20 [01:17<00:00,  3.87s/it]

0.3753947283799618


In [87]:
X[0].shape

(24, 210)

In [ ]:
x=np.random.rand(3,4)
np.exp(x)/np.exp(x).sum(1).reshape(3,-1)